# Tutorial 1 : Understand chess data processing

## Format of data

There are two ways to define a specific position in a chess game : 
* FEN format encode an entire position in one string.
* list of moves starting from the beginning position. (can be found in PGN files format)

### FEN format


In [ ]:
fen = "r1bq1rk1/2pp1ppp/p1n2n2/2b1p3/1pP1P3/1B1P1N2/PP3PPP/RNBQR1K1 b - c3 0 9" # Example of FEN
link_to_analyse_fen = "https://lichess.org/analysis"
### You should open Lichess and input this fen in the analysis tool (you should be able to reload a game from the given fen)

### List of moves

In [ ]:
from src.data_process.vocab import policy_index

move_vocab = policy_index # All move token are stored in policy index a list of all tokens possible 
print(f'Number of move token : {len(move_vocab)}, \n First tokens : {move_vocab[:50]}')

In [ ]:
from src.data_process.vocab import policy_index
moves = [1499,  187, 1221,  378, 1388,  319, 1060,  514, 1473,  134, 1694,  528,
        1533, 1139, 1661, 1784, 1204, 1592, 1480,  616, 1509,  891, 1473, 1375,
        1444, 1567,  749, 1496, 1193,    0,  506,  100, 1674,  980,  885,   81,
         785, 1495,  846,  289,  583,  551,  367,   63,  256,  875, 1413,   40,
        1206,  393,  931,  259,  726,  477,  942,  454, 1220,  726, 1486,  941,
        1685, 1187,  903,  559, 1338,  831,  658, 1174, 1487, 1107, 1717,  742,
        1742, 1349, 1709, 1846, 1473, 1754, 1206,  236,  929,  742,  667, 1285,
         906, 1508,  929, 1042,  658, 1434,  452,  474,  668,  276,  933,  783,
         903,   74,  667,  518,  903,  426,  668,  760,  940,   47, 1160,  964,
        1421,  918, 1188, 1855, 1446, 1775, 1253,   19, 1514,  760, 1691, 1397,
        1717,  974, 1743, 1511, 1768, 1249, 1791, 1379, 1768, 1568, 1791, 1791,
        1768, 1541, 1791, 1594, 1767, 1568, 1742, 1539, 1716, 1509, 1690, 1480,
        1664, 1504, 1638, 1307, 1614, 1095, 1638,  829, 1614, 1041, 1638, 1239,
        1614,  961, 1638,  715, 1614,  478, 1638,  519, 1614,  791, 1638,  826,
        1614,  861, 1638, 1104, 1614, 1123, 1638,  862, 1614, 1133, 1638, 1350,
        1614, 1597, 1638, 1791, 1614,  695, 1638, 1594, 1614, 1759, 1638,  941] # The output of the model after argmax for instance

for move in moves:
    print(policy_index[move])

## Preprocessing of data

### FEN for AI models

from a given FEN, we create a data representation of the game usable for AI models.

In [ ]:
from src.data_process.fen_encoder import fen_to_tensor

encoding = fen_to_tensor(fen)
print(f"encoded shape : {encoding.shape} \n It contains for each square a one hot representation of pieces + an encoding of available moves (like castle or en passant) \n {encoding[1,2]} \n IN ORDER : \n-\t piece present on the square **0-11**, \n-\t who to play (white or black) **12** \n-\t en passant available rule **13** \n-\t castling rights **14-17** \n-\t 50 move rules **18** ")

To use FEN as input data for our sequential models, we can use the FenEncoder class :

In [ ]:
import torch
from src.models.layers import FenEncoder
d_model = 512
model = FenEncoder(d_model)
inp = torch.from_numpy(encoding).unsqueeze(0)
out = model(inp)
print(out.shape)  #  shoud be [1, 64, 512] (batch size =1, seq_len = 64, encoding_dim = 512)

##

### Move list for AI models

We can just use list of move tokens for sequential models.

In [ ]:
# import torch
# from model import GPT,GPTConfig
# from data.parse import dir_iterator, ParsingConfig


# config = GPTConfig()
# config.vocab_size = 1929
# config.block_size = 256
# model = GPT(config).to("cuda")

# dir_path = "./pgn_data_example"
# parsing_config = ParsingConfig()
# parsing_config.batch_size=2
# parsing_config.padding_idx = config.vocab_size-1
# gen = dir_iterator(dir_path,parsing_config, return_fen=False)
# inp = next(gen)
# print(f"First elt of batch {inp[0].shape} corresponds to batch of encoded FEN, Second elt on batch corresponds to moves played after the position encoded as FEN : {inp[1].shape}. This list of moves is padded with a given index in parsing_config" )
# out = model(inp)[0]

# print(out.shape)